### Import das Bibliotecas

In [1]:
import os
import re
import librosa
import librosa.display

from sklearn.cluster import KMeans

import time
import pandas as pd
import numpy as np
import soundfile as sf

from itertools import combinations 

import IPython.display as ipd1
import matplotlib.pyplot as plt

from pathlib import Path

### Definição de Constantes

In [2]:
FFT_FRAME_SIZE = 2048
HOP_LENGTH = 1024

### Carregamento do Dataframe com os Diretórios dos Áudios

In [3]:
dsPath = '.\\Datasets\\Projeto\\'
dsInfo = pd.read_csv(dsPath+'DiretoriosEventos.csv',sep=';') 
dsInfo

,Diretorio,Evento
0,BoltAction22,Disparo Bolt
1,CracklingFire,Incêndio
2,GlassBreaking,Vidro Quebrando
3,WaterDrops,Água Pingando


### Carregamento dos Arquivos de Áudios

In [4]:
count = 0
tamMaiorAudio =0
qtdArqSubDir = list()
allArqAudio = list()

print('inicio')
inicio = time.time()

for subdir in dsInfo['Diretorio']:
    dirPath = Path(dsPath+'\\Audios\\'+subdir)
    arqObjectsPath = list(dirPath.glob('**/*.wav'))
    qtdArqSubDir.append(len(arqObjectsPath))
    
    for x in arqObjectsPath:
        count = count + 1
        audSignal, audSR = librosa.load(x, sr=44100)
        allArqAudio.append([subdir, x, audSignal.tolist(), len(audSignal)])
        
print(count)
fim = time.time()
print('fim - duração {0}'.format(fim-inicio))
    
dsInfo['QtdArq'] = qtdArqSubDir

inicio
506
fim - duração 1.2486176490783691


### Conversão da Lista de Áudios em Dataframe

In [5]:
allArqAudioDF = pd.DataFrame(allArqAudio,columns=['diretorio','path','audio','tamanho'])
allArqAudioDF.groupby(['diretorio']).count()

,path,audio,tamanho
diretorio,,,
BoltAction22,142,142,142
CracklingFire,102,102,102
GlassBreaking,151,151,151
WaterDrops,111,111,111


### Declaração da Função Geradora do Espectrograma Mel

In [6]:
def geraSpectrograma(audioSignal, sampleRate):
    mel_spectrogram = librosa.feature.melspectrogram(audioSignal, sr=sampleRate, n_fft=FFT_FRAME_SIZE, 
                                                      hop_length=HOP_LENGTH, n_mels=10)
    log_mel_spectrogram = librosa.power_to_db(mel_spectrogram)
    
    return log_mel_spectrogram

### Declaração da Função que Salva o Espectrograma em Arquivo

In [7]:
def salvaSpectrograma(nomeSemExt, spectro):
    arqSpectro = str(nomeSemExt)
    arqSpectro = arqSpectro.replace('Audios','Spectrogramas')
    arqSpectro = arqSpectro + '.npy'

    tempPath = Path(arqSpectro)
    if not tempPath.parents[0].exists():
        tempPath.parents[0].mkdir(parents=True)
    np.save(arqSpectro, spectro) 

### Declaração da Função que Salva o Novo Segmento de Áudio em Arquivo

In [8]:
def salvaAudioSegmento(nomeSemExt, segment, rate):
    arqAudio = str(nomeSemExt)
    arqAudio = arqAudio + '.wav'

    os.makedirs(os.path.dirname(arqAudio), exist_ok=True)
    with sf.SoundFile(arqAudio, 'w', samplerate=rate, channels=1, subtype='PCM_16') as f:
        f.write(segment)

### Declaração da Função que Gera os Mixes de Áudios por Combinação  

In [9]:
def combinaAudiosDiretorios(combinacao, dirInfo, arqAudioDF):
    qtdComb = len(combinacao)
    qtdAudMixed = min(dirInfo[dirInfo['Diretorio'].isin(combinacao)]['QtdArq'])
    
    audioSeries = np.array([arqAudioDF[arqAudioDF['diretorio'].isin([x])]['audio'].iloc[0:qtdAudMixed].tolist() for x in combinacao])
    audioMixed = np.sum(audioSeries, axis=0) / qtdAudMixed

    arqNameBase = ''.join([x[:3] for x in combinacao])
    dirName = '_'.join(combinacao)
    dirPath = arqAudioDF[arqAudioDF['diretorio'].isin([combinacao[0]])]['path'].iloc[0]
    dirPath = str(dirPath).replace(combinacao[0], dirName)
    dirPath = dirPath[0:dirPath.rindex('\\')+1]
                         
    for x in range(0, qtdAudMixed):
        arqNome = dirPath+arqNameBase+str(x+1)
        spectrograma = geraSpectrograma(audioMixed[x], 44100)
        salvaSpectrograma(arqNome, spectrograma)
        salvaAudioSegmento(arqNome, audioMixed[x], 44100)            
        

### Loop que Gera as Combinações e Aciona a Função Geradora de Áudios Mixados

In [10]:
#Combinatória dos Audios para Detecção Multi Label

print('inicio')
inicio = time.time()

tamDF = len(dsInfo['Diretorio'])
for x in range(2, tamDF+1):
    combos = combinations(dsInfo['Diretorio'], x)
    for comb in list(combos): 
        print(comb)
        combinaAudiosDiretorios(comb, dsInfo, allArqAudioDF)
        
fim = time.time()
print('fim - duração {0}'.format(fim-inicio))

inicio
('BoltAction22', 'CracklingFire')
('BoltAction22', 'GlassBreaking')
('BoltAction22', 'WaterDrops')
('CracklingFire', 'GlassBreaking')
('CracklingFire', 'WaterDrops')
('GlassBreaking', 'WaterDrops')
('BoltAction22', 'CracklingFire', 'GlassBreaking')
('BoltAction22', 'CracklingFire', 'WaterDrops')
('BoltAction22', 'GlassBreaking', 'WaterDrops')
('CracklingFire', 'GlassBreaking', 'WaterDrops')
('BoltAction22', 'CracklingFire', 'GlassBreaking', 'WaterDrops')
fim - duração 22.007357835769653
